In [94]:
from functools import reduce
from itertools import combinations
# from queue import PriorityQueue


import numpy as np
from scipy.optimize import milp, LinearConstraint

In [ ]:
# from io import StringIO
# IN_ = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
# [...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
# [.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""
# IN = StringIO(IN_)
IN = open("d10_in.txt")
with IN as f:
    lines = f.read().split("\n")
    lines = [line.split(" ") for line in lines]
    # only for P1
    RESULTS = [line[0][::-1].strip("[]") for line in lines]
    RESULTS = [r.replace(".", "0").replace("#", "1") for r in RESULTS]
    RESULTS_INT = [int(r, 2) for r in RESULTS]
    # for p2, meaning of summands changes
    SUMMANDS = [line[1:-1] for line in lines]
    SUMMANDS = [[s.strip("()").split(",") for s in summand] for summand in SUMMANDS]
    SUMMANDS = [
        tuple(
            tuple([int(x) for x in term])
            for term in summand
        )
        for summand in SUMMANDS
    ]
    SUMMANDS_INT = [
        [sum(2**x for x in term) for term in summand]
        for summand in SUMMANDS
    ]
    # P2 only
    TOTALS = [line[-1].strip("{}").split(",") for line in lines]
    TOTALS = [tuple([int(t) for t in total]) for total in TOTALS]

Part 1

In [ ]:
mins = []
for i in range(len(RESULTS)):
    min_ = 1_000_000
    found_min = False
    r = RESULTS_INT[i]
    summands = SUMMANDS_INT[i]
    n_bits = len(RESULTS[i])
    for n in range(1, len(summands) + 1):
        for comb in combinations(summands, n):
            if reduce(lambda x, y:x^y, comb, 0) == r:
                min_ = n
                found_min = True
                break
        if found_min:
            break
    mins.append(min_)
sum(mins)

Part 2

In [97]:
TOTALS_NP = [np.array(t).astype(np.int32) for t in TOTALS]
LENGTHS = [len(t) for t in TOTALS]
DELTAS_LIST_NP = []
for i in range(len(LENGTHS)):
    el = LENGTHS[i]
    options = SUMMANDS[i]
    DELTAS = [np.zeros(el) for _ in options]
    for j, o in enumerate(options):
        for t in o:
            DELTAS[j][t] = 1
    DELTAS_NP = [x.astype(np.int32) for x in DELTAS]
    DELTAS_LIST_NP.append(DELTAS_NP)

In [98]:
# def get_nbhs(v: np.ndarray, deltas: list[np.ndarray]) -> list[np.ndarray]:
#     return [
#         tuple((v - d).tolist())
#         for d in deltas
#         if ((v - d) >= 0).all()
#     ]
# def dijkstra(source, deltas, dist={}):
#     source_ = tuple(source.tolist())
#     dist[source_] = 0
#     prev = {source_: None}
#     Q = PriorityQueue()
#     Q.put((dist[source_], source_))
#     BIG_NUM = 1_000_000_000
#     found = False

#     while not Q.empty():
#         du, u = Q.get()

#         for v in get_nbhs(u, deltas):
#             alt = du + 1
#             if alt < (BIG_NUM if v not in dist else dist[v]):
#                 dist[v] = alt
#                 prev[v] = u
#                 if (np.array(v).astype(np.int32) == 0).all():
#                     found = True
#                     break
#                 elif (np.array(v).astype(np.int32) < 0).any():
#                     continue
#                 else:
#                     Q.put((dist[v], v))
#         if found:
#             break

In [99]:
optims = []

# scipy optimize mixed integer linear programming
for i in range(len(TOTALS_NP)):
    b_l, b_u = TOTALS_NP[i], TOTALS_NP[i]
    c = np.ones(len(DELTAS_LIST_NP[i])).astype(np.int32)
    A = np.array([a.tolist() for a in DELTAS_LIST_NP[i]]).T
    constr = LinearConstraint(A, b_l, b_u)
    integrality = np.ones_like(c)
    res = milp(c=c, constraints=constr, integrality=integrality)
    num = res.x.sum().item()
    optims.append(num)

In [ ]:
sum(optims)